In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pandas as pd
import numpy as np
import attila_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
fpath = '../../results/2022-01-14-rank-aggregation/aggregated-ranks.csv'
ranks = pd.read_csv(fpath, index_col='rank')
ranks

,mean,median,geo.mean,l2norm,MC1,MC2,MC3
rank,,,,,,,
1,CHEMBL446452,CHEMBL296306,CHEMBL1318287,CHEMBL446452,CHEMBL1193,CHEMBL1318287,CHEMBL491473
2,CHEMBL296306,CHEMBL1193,CHEMBL1139,CHEMBL296306,CHEMBL296306,CHEMBL1139,CHEMBL477772
3,CHEMBL1193,CHEMBL446452,CHEMBL446452,CHEMBL1193,CHEMBL1139,CHEMBL446452,CHEMBL446452
4,CHEMBL1139,CHEMBL46257,CHEMBL46257,CHEMBL46257,CHEMBL446452,CHEMBL46257,CHEMBL296306
5,CHEMBL46257,CHEMBL1139,CHEMBL1121,CHEMBL1139,CHEMBL46257,CHEMBL1193,CHEMBL1193
...,...,...,...,...,...,...,...
601,CHEMBL1484251,CHEMBL1201161,CHEMBL1484251,CHEMBL1484251,CHEMBL1200370,CHEMBL193240,CHEMBL54349
602,CHEMBL2087361,CHEMBL1200332,CHEMBL2087361,CHEMBL2087361,CHEMBL1484251,CHEMBL3707348,CHEMBL2220442
603,CHEMBL1201153,CHEMBL1200659,CHEMBL1201153,CHEMBL1201153,CHEMBL13766,CHEMBL1076347,CHEMBL1200370


In [3]:
fpath = '../../results/2021-12-13-chembl-drug-info/drug-indication.csv'
indications = pd.read_csv(fpath, index_col='drug_chembl_id')
# D000544 is Alzheimer disease
mp4ad = indications.loc[indications.mesh_id == 'D000544', 'max_phase_for_ind']

In [4]:
fpath = '../../results/2021-12-02-proximity-various-ADgenesets/merged-proximity-chembl.csv'
usecols = range(6)
#usecols = ['', 'drug_name', 'max_phase', 'indication_class', 'uniprot_name', 'target_name']
merged = pd.read_csv(fpath, index_col=0, skiprows=1, usecols=usecols).rename_axis('drug_chembl_id', axis=0)

In [5]:
top_drugs = pd.DataFrame({'rank': ranks.index}, index=ranks.MC3).rename_axis('drug_chembl_id', axis=0)
top_drugs = pd.merge(top_drugs, merged, how='left', on='drug_chembl_id')
top_drugs = pd.merge(top_drugs, mp4ad, how='left', on='drug_chembl_id').rename({'max_phase_for_ind': 'max_phase_for_AD'}, axis=1)
top_drugs

,rank,drug_name,max_phase,indication_class,uniprot_name,target_name,max_phase_for_AD
drug_chembl_id,,,,,,,
CHEMBL491473,1,CEDIRANIB,3,NaN,CDC7|PK3CD|EF2K|PLK4|STK25|P3C2B|RIOK3|MP2K7|I...,Cell division cycle 7-related protein kinase|P...,NaN
CHEMBL477772,2,PAZOPANIB,4,NaN,BMR1B|PK3CD|PLK4|STK25|P3C2B|RIOK3|MP2K7|CHK1|...,Bone morphogenetic protein receptor type-1B|PI...,NaN
CHEMBL446452,3,ARUNDINE,3,NaN,RGS4|GPR84,Regulator of G-protein signaling 4|G-protein c...,NaN
CHEMBL296306,4,LACTULOSE,4,Laxative,RGS4,Regulator of G-protein signaling 4,NaN
CHEMBL1193,5,PHENIRAMINE,4,NaN,RGS4,Regulator of G-protein signaling 4,NaN
...,...,...,...,...,...,...,...
CHEMBL54349,601,ALPIDEM,4,Anti-Anxiety Agent,ABCBB|ALBU|TSPO|TYDP1,Bile salt export pump|Serum albumin|Translocat...,NaN
CHEMBL2220442,602,FLUVASTATIN,4,Antihyperlipidemic; Inhibitor (HMG-CoA reductase),HMDH|CP2C9,HMG-CoA reductase|Cytochrome P450 2C9,NaN
CHEMBL1200370,603,BENZOYL PEROXIDE,4,Keratolytic,LMNA|THB,Prelamin-A/C|Thyroid hormone receptor beta-1,NaN


In [6]:
%connect_info

{
  "shell_port": 52124,
  "iopub_port": 52125,
  "stdin_port": 52126,
  "control_port": 52128,
  "hb_port": 52127,
  "ip": "127.0.0.1",
  "key": "837b22aa-6284b7c393e08a165b23190e",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-9dc2acaa-dd9e-47f8-8473-80bf157cbe8e.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
